# A Behavioral-Economic Analysis of Demand for Social and Food Reinforcement in Rats

This notebook replicates the own- and cross-price demand curve analyses from the publication:

> Kirkman, C., Wan, H., & Hackenberg, T. D. (2022). A behavioral-economic analysis of demand and preference for social and food reinforcement in rats. *Learning and Motivation*, *77*, 101780. https://doi.org/10.1016/j.lmot.2021.101780

The analysis fits nonlinear and linear models to experimental data to quantify demand for food and social reinforcers in rats. This script uses `pandas` for data manipulation and `scipy` for nonlinear model fitting.

In [34]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from lmfit import Model, Parameters
import warnings

# Suppress warnings from curve_fit, which can be verbose
warnings.filterwarnings("ignore")

# Set the display format for floating-point numbers to 3 decimal places
pd.options.display.float_format = '{:.3f}'.format

# --- Helper Functions (translated from R) ---
def lhs(x):
    """Inverse Hyperbolic Sine (log-like) transformation."""
    return np.log10(0.5 * x + np.sqrt(0.25 * (x**2) + 1))

# --- Load Data ---
behav_data = pd.read_csv("Data/data with bl mean.csv")
# Add transformed consumption columns for modeling
behav_data['foodr_lhs'] = lhs(behav_data['foodr'])
behav_data['socr_lhs'] = lhs(behav_data['socr'])

In [35]:
# --- Define the behavioral-economic model functions ---

def zbe_model(price, q0, alpha):
    """
    Zero-Bounded Exponential (ZBEn) model for own-price demand.
    Note: The function is defined in terms of the transformed `lhs` space.
    """
    return lhs(q0) * np.exp(((-alpha) / lhs(q0)) * q0 * price)

def cross_price_exp_model(price, q_alone, i, b):
    """
    Exponential cross-price elasticity model.
    Note: This models raw consumption, not the lhs-transformed version.
    """
    return np.log10(q_alone) + i * np.exp(-b * price)

## Phase 1: Food Price Varied, Social Price Constant

In this phase, we analyze own-price demand for food and cross-price demand for social interaction.

In [36]:
# --- Own-Price Food Demand (ZBEn Model) ---
print("--- Phase 1: Own-Price Food Demand (ZBEn Model) ---")
zbe_model_obj = Model(zbe_model, independent_vars=['price'])
p1_data = behav_data[behav_data['cond'] == 1]
results_list_own_p1 = []

for subj_id, df in p1_data.groupby('subj'):
    params = Parameters()
    params.add('q0', value=50, min=0)
    params.add('alpha', value=0.0001, min=0)
    
    result = zbe_model_obj.fit(df['foodr_lhs'], params, price=df['foodfr'])
    fit_params = result.best_values
    results_list_own_p1.append({'subj': subj_id, **fit_params})

own_price_food_p1_lmfit = pd.DataFrame(results_list_own_p1)
print(own_price_food_p1_lmfit)


# --- Cross-Price Social Demand (Exponential Model) ---
print("\n--- Phase 1: Cross-Price Social Demand (Exponential Model) ---")
exp_model_obj = Model(cross_price_exp_model, independent_vars=['price'])
results_list_cross_p1 = []

for subj_id, df in p1_data.groupby('subj'):
    params = Parameters()
    params.add('q_alone', value=100, min=0)
    params.add('i', value=1)
    params.add('b', value=0.01, min=0)
    
    result = exp_model_obj.fit(df['socr'], params, price=df['foodfr'])
    fit_params = result.best_values
    results_list_cross_p1.append({'subj': subj_id, 'Q_alone': fit_params['q_alone'], 'I': fit_params['i'], 'beta': fit_params['b']})

cross_price_social_p1_lmfit = pd.DataFrame(results_list_cross_p1)
print(cross_price_social_p1_lmfit)

--- Phase 1: Own-Price Food Demand (ZBEn Model) ---
   subj      q0  alpha
0     1 177.745  0.000
1     2 260.448  0.000
2     3 393.347  0.000
3     4 217.576  0.000

--- Phase 1: Cross-Price Social Demand (Exponential Model) ---
   subj                                           Q_alone       I  \
0     1                                        231454.270  31.369   
1     2                135935571189553077440617644032.000 -63.011   
2     3 912010838418402909757689809298942069607235584.000 249.541   
3     4                                     583665967.630  16.609   

              beta  
0            0.000  
1 319044590961.900  
2          248.734  
3            0.000  


## Phase 2: Social Price Varied, Food Price Constant

Here, we analyze own-price demand for social interaction and cross-price demand for food.

In [37]:
print("--- Phase 2: Own-Price Social Demand (ZBEn Model) ---")
p2_data = behav_data[behav_data['cond'] == 2]
results_list_own_p2 = []

for subj_id, df in p2_data.groupby('subj'):
    params = Parameters()
    params.add('q0', value=50, min=0)
    params.add('alpha', value=0.0001, min=0)
    
    result = zbe_model_obj.fit(df['socr_lhs'], params, price=df['socfr'])
    fit_params = result.best_values
    results_list_own_p2.append({'subj': subj_id, **fit_params})

own_price_social_p2_lmfit = pd.DataFrame(results_list_own_p2)
print(own_price_social_p2_lmfit)


print("\n--- Phase 2: Cross-Price Food Demand (Exponential Model) ---")
results_list_cross_p2 = []

for subj_id, df in p2_data.groupby('subj'):
    params = Parameters()
    params.add('q_alone', value=10000)
    params.add('i', value=100)
    params.add('b', value=0.1)
    
    result = exp_model_obj.fit(df['foodr'], params, price=df['socfr'])
    fit_params = result.best_values
    results_list_cross_p2.append({'subj': subj_id, 'Q_alone': fit_params['q_alone'], 'I': fit_params['i'], 'beta': fit_params['b']})

cross_price_food_p2_lmfit = pd.DataFrame(results_list_cross_p2)
print(cross_price_food_p2_lmfit)

--- Phase 2: Own-Price Social Demand (ZBEn Model) ---
   subj     q0  alpha
0     1 36.600  0.004
1     2 67.848  0.003
2     3 15.308  0.011
3     4 48.438  0.004

--- Phase 2: Cross-Price Food Demand (Exponential Model) ---
   subj                                            Q_alone       I   beta
0     1 48903866479660579142642000455998361332897238701...  15.598 -0.074
1     2                                       51667831.075 196.441 -0.000
2     3 15727811058958733523221124004665647997409225195...  55.178  0.437
3     4 13989955337764378763772905330157635119202573126...  22.842 -0.100


## Phase 3: Concurrent Price Increase

In this phase, the prices of both reinforcers increased together. We fit own-price demand curves for both food and social interaction.

In [38]:
print("--- Phase 3: Own-Price Food Demand (ZBEn Model) ---")
p3_data = behav_data[behav_data['cond'] == 3]
results_list_food_p3 = []

for subj_id, df in p3_data.groupby('subj'):
    params = Parameters()
    params.add('q0', value=50, min=0)
    params.add('alpha', value=0.0001, min=0)
    
    result = zbe_model_obj.fit(df['foodr_lhs'], params, price=df['foodfr'])
    fit_params = result.best_values
    results_list_food_p3.append({'subj': subj_id, **fit_params})

own_price_food_p3_lmfit = pd.DataFrame(results_list_food_p3)
print(own_price_food_p3_lmfit)


print("\n--- Phase 3: Own-Price Social Demand (ZBEn Model) ---")
results_list_social_p3 = []

for subj_id, df in p3_data.groupby('subj'):
    params = Parameters()
    params.add('q0', value=50, min=0)
    params.add('alpha', value=0.0001, min=0)
    
    result = zbe_model_obj.fit(df['socr_lhs'], params, price=df['socfr'])
    fit_params = result.best_values
    results_list_social_p3.append({'subj': subj_id, **fit_params})

own_price_social_p3_lmfit = pd.DataFrame(results_list_social_p3)
print(own_price_social_p3_lmfit)

--- Phase 3: Own-Price Food Demand (ZBEn Model) ---
   subj      q0  alpha
0     2 159.777  0.000
1     3 195.924  0.000

--- Phase 3: Own-Price Social Demand (ZBEn Model) ---
   subj     q0  alpha
0     2 64.046  0.001
1     3 55.452  0.005


## Phase 4: Food Price Varied (Alone)

In this final phase, food price was varied without a concurrent social alternative. This allows for a direct comparison of food demand elasticity in the presence versus absence of a substitute.

In [39]:
print("--- Phase 4: Own-Price Food Demand (ZBEn Model, No Social Alternative) ---")
p4_data = behav_data[behav_data['cond'] == 4]
results_list_p4 = []

for subj_id, df in p4_data.groupby('subj'):
    params = Parameters()
    params.add('q0', value=50, min=0)
    params.add('alpha', value=0.0001, min=0)
    
    result = zbe_model_obj.fit(df['foodr_lhs'], params, price=df['foodfr'])
    fit_params = result.best_values
    results_list_p4.append({'subj': subj_id, **fit_params})

own_price_food_p4_lmfit = pd.DataFrame(results_list_p4)
print(own_price_food_p4_lmfit)

--- Phase 4: Own-Price Food Demand (ZBEn Model, No Social Alternative) ---
   subj      q0  alpha
0     2 273.148  0.000
1     3 193.248  0.000
